----
1 – Crie uma estrutura de dados em Python para suportar o conjunto de treino. Lembre-se que o
conjunto de treino deve conter, para cada exemplo, a classe respetiva numa forma que seja adequada
à implementação de uma rede neuronal. Embora esta questão, nesta altura, lhe possa parecer um
pouco estranha, mais adiante virá a compreender este requisito.

In [1]:
import pandas as pd
import os

os.chdir('C:\\Users\\Rafael\\Desktop\\4º Ano\\2º Semestre\\PS\\Aulas Práticas')

data=pd.read_excel('6 - CT 2018-19-1.xlsx')

data

,x,y,classe
0,-6.760254,-2.438752,0
1,0.237898,-8.227674,0
2,2.009494,0.624965,0
3,-9.850935,0.177774,0
4,-1.074828,-8.673791,0
5,-1.196868,8.340757,0
6,7.636788,9.787538,0
7,-0.631973,5.123220,0
8,-8.275424,-2.379850,0
9,-0.198645,-8.716467,0


----
2 – Para que uma rede neuronal seja facilmente configurável, possibilitando alterar de uma forma fácil
o número de camadas, o número de neurónios em cada camada e as ligações entre neurónios, é
necessário criar uma forma de a descrever que possibilite desenvolver software que, baseado nessa
descrição, implemente adequadamente a rede. Crie uma descrição que lhe pareça adequada ao
propósito apresentado de forma a poder iniciar o desenvolvimento do seu software.

In [2]:
#Descrição da Rede Neuronal feita por dicionários

NN={'Input':{'I0':{'Out':0},'I1':{'Out':0}},
    'Layer0':{'N0':{'Out':0, 'Bias':0.4,'I0':1.4,'I1':1.0},
              'N1':{'Out':0, 'Bias':-1.5,'I0':1.0,'I1':-0.5}},
    'Layer1':{'N2':{'Out':0, 'Bias':5.4,'N0':-8.0,'N1':-10}}
   }

In [3]:
NN['Layer0']['N0']

{'Out': 0, 'Bias': 0.4, 'I0': 1.4, 'I1': 1.0}

----
3 – Comece por calcular a chamada rede forward. Tenha em conta que com os pesos:

W10=0,4 W11=1,4 W12=1,0

W20=-1,5 W21=1,0 W22=-0,5

W20=5,4 W21=-8,0 W22=-10,0  

Saída=0

Deverá obter:

Soma=-11,5031078998 Sigmoide=0,0000100986
 
Soma=-7,0408777940  Sigmoide=0,0008745923

Soma=5,3911732889 Sigmoide=0,9954640447
 

In [4]:
#Introduzir os valores pretendidos para x e y
NN['Input']['I0']['Out']=data.iloc[0][0]
NN['Input']['I1']['Out']=data.iloc[0][1]

In [5]:
NN

{'Input': {'I0': {'Out': -6.760253967}, 'I1': {'Out': -2.438752346}},
 'Layer0': {'N0': {'Out': 0, 'Bias': 0.4, 'I0': 1.4, 'I1': 1.0},
  'N1': {'Out': 0, 'Bias': -1.5, 'I0': 1.0, 'I1': -0.5}},
 'Layer1': {'N2': {'Out': 0, 'Bias': 5.4, 'N0': -8.0, 'N1': -10}}}

In [6]:
from math import exp
def getNNval(pLayer,ni):
    if ni=='Bias':
        return (1)
    return(NN[pLayer][ni]['Out'])


def sigmoidCalc(NN):
    prevLayer=-1
    for n,l in enumerate(NN):
        if n==0:
            prevLayer=l
            continue
        for n in NN[l]:
           #print('Neuronio:',n)
            sum=0;
            for i in NN[l][n]:
                if i=='Out':
                    continue
    #            print(NN[l][n][i])
               # print('Layer',prevLayer,i)
                sum +=(NN[l][n][i])*getNNval(prevLayer,i)
            #print('Sigmoid:', 1/(1+exp(-sum)))
            NN[l][n]['Out']=1/(1+exp(-sum))
            #print('Soma: ',sum)
        prevLayer=l
        
sigmoidCalc(NN)

----
4 – Calcule agora os deltas correspondentes ao exemplo anterior. Os valores para este exemplo
deverão ser:

Delta 1: 0,0000003631 

Delta 2: 0,0000392777 

Delta 3: -0,0044948989


O Fator de aprendizagem é 0,1

In [7]:
def getOutValues(plev,neu,dlist):
    res=0
    for n in NN[plev]:
        if neu in NN[plev][n]:
            res+=NN[plev][n][neu]*dlist[n]
    return res

#target=data.iloc[0]['classe']
def calcDeltas(NN,target):
    levels=[lev for lev in NN]
    levels.reverse()

    deltaList={}
    prevLevel=-1

    for n,level in enumerate(levels[:-1]):
        for neuron in NN[level]:
            if n==0: #Saída!!
                out=NN[level][neuron]['Out']
                
                delta=out*(1-out)*(target-out)
            else:
                sumnn=getOutValues(prevLevel,neuron,deltaList)
                out=NN[level][neuron]['Out']
                target=data.iloc[0]['classe']
                delta=out*(1-out)*sumnn
            deltaList[neuron]=delta
        prevLevel=level
    return deltaList

In [8]:
calcDeltas(NN,data.iloc[0]['classe'])


{'N2': -0.004494898864430347,
 'N0': 3.6313227823143915e-07,
 'N1': 3.927765555330235e-05}

----
5 – Faça agora a alteração dos pesos sinápticos de acordo com os deltas encontrados anteriormente
e usando um fator de aprendizagem de 0.1. Os novos pesos deverão ser:

W10=0,4000000363 w11=1,3999997545 w12=0,9999999114
 
W20=-1,4999960722 w21=0,9999734473 w22=-0,5000095788
  
W30=5,3995505101 w31=-8,0000000045 w32=-10,0000003931
  

In [9]:
def adjustWeights(NN,deltaList):
    for i,lev in enumerate(NN):
        if i==0: #Entradas, não são para alterar
            prevLevel=lev
            continue
        
        for neu in NN[lev]: #percorre cada neurónio
            #print('Neuronio:',neu)
            for j,weight in enumerate(NN[lev][neu]):
               # print(weight)
                
                if weight=='Out':
                    continue
                if weight=='Bias':
                    NN[lev][neu][weight] +=0.1*deltaList[neu]*1
                    #print(NN[lev][neu][weight])
                    continue
                NN[lev][neu][weight] += 0.1 * deltaList[neu] * NN[prevLevel][weight]['Out']
                #print(NN[lev][neu][weight])
        prevLevel=lev
    

deltaList=calcDeltas(NN,data.iloc[0]['classe'])
adjustWeights(NN,deltaList)

----
6 – Aplique o processo anterior a todo o conjunto de treino efetuando aqui que se chama uma “época”.
Os pesos no final da primeira época deverão ser:


W10=0,3762664999 w11=1,4192249866 w12=0,9594166897
  
W20=-1,5049663822 w21=1,0018069293 w22=-0,5105108826
  
W30=5,4111086566 w31=-7,9902423128 w32=-9,9994048385
  

In [46]:
NN={'Input':{'I0':{'Out':0},'I1':{'Out':0}},
    'Layer0':{'N0':{'Out':0, 'Bias':0.4,'I0':1.4,'I1':1.0},
              'N1':{'Out':0, 'Bias':-1.5,'I0':1.0,'I1':-0.5}},
    'Layer1':{'N2':{'Out':0, 'Bias':5.4,'N0':-8.0,'N1':-10}}
   }

for n in range(0,len(data),1):
#Introduzir os valores pretendidos para x e y
    NN['Input']['I0']['Out']=data.iloc[n][0]
    NN['Input']['I1']['Out']=data.iloc[n][1]
    sigmoidCalc(NN)
    deltaList=calcDeltas(NN,data.iloc[n]['classe'])
    adjustWeights(NN,deltaList)
print(NN['Layer0']['N0'])
print(NN['Layer0']['N1'])
print(NN['Layer1']['N2'])


{'Out': 0.00876826440991753, 'Bias': 0.376266499879864, 'I0': 1.4192249866391529, 'I1': 0.9594166897413322}
{'Out': 7.4378803064963155e-06, 'Bias': -1.5049663822405734, 'I0': 1.0018069292685343, 'I1': -0.5105108826002814}
{'Out': 0.9952316323029436, 'Bias': 5.411108656628424, 'N0': -7.9902423127876565, 'N1': -9.999404838452232}


In [37]:
data

,x,y,classe
0,-6.760254,-2.438752,0
1,0.237898,-8.227674,0
2,2.009494,0.624965,0
3,-9.850935,0.177774,0
4,-1.074828,-8.673791,0
5,-1.196868,8.340757,0
6,7.636788,9.787538,0
7,-0.631973,5.123220,0
8,-8.275424,-2.379850,0
9,-0.198645,-8.716467,0
